### Collate XLSSC122 and 3DHST galfit catalogs
- Dec 2/2020

### Load modules

In [25]:
from __future__ import division
import sys
sys.path.insert(0,'/data/emiln/XLSSC122_GalPops/Analysis/Modules')
from GalfitPyWrap import galfitwrap as gf
import galfit_helpers as gfh
import os
import numpy as np
import pandas as pd
import glob
from astropy import units as u
from astropy import wcs
from astropy.coordinates import SkyCoord
from astropy.io import fits, ascii
from astropy.table import Table, hstack, join
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### HST

In [58]:
# Load catalogs
galfit_filename = '/data/emiln/XLSSC122_GalPops/Analysis/HST/Results/final_galfit_stellar_PSF.csv'
# phot_filename =  '/data/emiln/XLSSC122_GalPops/Data/HST/Products/catalogs/xlssc122_F105_F140_redshifts_short_v4_snr_em_member2_by_hand.cat'

HST_cat_root = '/data/emiln/XLSSC122_GalPops/Data/HST/Products/catalogs/'
HST_cat_filename = HST_cat_root+'xlssc122_F105_F140_redshifts_short_v4_snr_em_member2_by_hand.cat'
full_df, phot_df = gfh.load_HST_galaxy_catalog(HST_cat_filename)

galfit_df = pd.read_csv(galfit_filename)
# phot_df = pd.read_csv(phot_filename, delim_whitespace=True)

In [59]:
phot_df.columns.values

array(['phot-id', 'ra', 'dec', 'f105_kron', 'f105_ekron', 'f105_0p8',
       'f105_e0p8', 'f140_kron', 'f140_ekron', 'f140_0p8', 'f140_e0p8',
       'asec_fwhm', 'zz', 'zlo', 'zhi', 'chi', 'spec-id', 'mcode', 'snr',
       'em-code', 'member1', 'member2', 'gold_cluster', 'gold_foreground',
       'silver_cluster', '(f140)kron', 'color', 'F140W', 'ID', 'RA',
       'DEC', 'r_center', 'X', 'Y'], dtype=object)

In [60]:
# Merge
cols_to_use = phot_df.columns.difference(galfit_df.columns)
hst_mdf = galfit_df.merge(phot_df[cols_to_use],how='inner',left_on='ID',right_on='phot-id')
print(hst_mdf.columns.values)
print(len(hst_mdf))
print(len(galfit_df))

['ID' 'ra' 'dec' 're' 're_err' 'ar' 'ar_err' 'n' 'n_err' 'mag' 'mag_err'
 'sky' 'sky_err' 'chi2nu' 'ErrorValue' '(f140)kron' 'DEC' 'F140W' 'RA' 'X'
 'Y' 'asec_fwhm' 'chi' 'color' 'em-code' 'f105_0p8' 'f105_e0p8'
 'f105_ekron' 'f105_kron' 'f140_0p8' 'f140_e0p8' 'f140_ekron' 'f140_kron'
 'gold_cluster' 'gold_foreground' 'mcode' 'member1' 'member2' 'phot-id'
 'r_center' 'silver_cluster' 'snr' 'spec-id' 'zhi' 'zlo' 'zz']
37
37


In [61]:
### Compute stellar mass statistics from chains
files_mstar = glob.glob('/data/emiln/XLSSC122_GalPops/Data/Products/HST/catalogs/mass_chains/*_post_lin_tw_mstar.out')

In [62]:
ids = []
masses = []
stds = []
for f in files_mstar:
    mstar = pd.read_csv(f,header=None,skiprows=1,delimiter='\s+')
    id=f.split('/')[-1].split('_')[0]
    if id[0] == '0': id=id[1:]
    x=mstar[0]; y=mstar[1]; z=mstar[2]
    nx=101
    xu = np.unique(x)
    yu = np.unique(y)
    zu = np.reshape(z,(nx,nx))
    p = np.sum(zu,axis=0)
    p_norm = p/np.sum(p)
    # print(len(p_norm),len(xu))
    mass = np.dot(xu,p_norm)
    std = np.sqrt(np.dot((xu-mass)**2,p_norm))
    ids.append(int(id))
    masses.append(float(mass))
    stds.append(float(std))



In [63]:
mass_df

,spec-id,mass,mass_err
0,240,10.231026,0.137707
1,295,10.968724,0.013495
2,298,10.822529,0.076894
3,345,9.983817,0.208212
4,428,10.004328,0.267618
5,451,10.755394,0.087396
6,493,10.249928,0.169805
7,526,11.619915,0.040020
8,606,10.308117,0.118456
9,649,10.857841,0.102053


In [64]:
mass_df = pd.DataFrame(list(zip(ids,masses,stds)),columns=['spec-id','mass','mass_err'])
mhst_mdf = hst_mdf.merge(mass_df,on='spec-id',how='outer')
print(mhst_mdf.columns.values)
mhst_mdf

['ID' 'ra' 'dec' 're' 're_err' 'ar' 'ar_err' 'n' 'n_err' 'mag' 'mag_err'
 'sky' 'sky_err' 'chi2nu' 'ErrorValue' '(f140)kron' 'DEC' 'F140W' 'RA' 'X'
 'Y' 'asec_fwhm' 'chi' 'color' 'em-code' 'f105_0p8' 'f105_e0p8'
 'f105_ekron' 'f105_kron' 'f140_0p8' 'f140_e0p8' 'f140_ekron' 'f140_kron'
 'gold_cluster' 'gold_foreground' 'mcode' 'member1' 'member2' 'phot-id'
 'r_center' 'silver_cluster' 'snr' 'spec-id' 'zhi' 'zlo' 'zz' 'mass'
 'mass_err']


,ID,ra,dec,re,re_err,ar,ar_err,n,n_err,mag,...,phot-id,r_center,silver_cluster,snr,spec-id,zhi,zlo,zz,mass,mass_err
0,529,34.434215,-3.758796,96.2689,11.8374,0.8192,0.0056,7.5569,0.2702,19.7563,...,529,0.010969,0,34.08530,526,1.981394,1.975346,1.979632,11.619915,0.040020
1,455,34.422282,-3.763513,4.6240,0.0721,0.9368,0.0087,2.9442,0.0643,21.7323,...,455,1.313392,0,22.31920,451,1.981299,1.974240,1.980749,10.755394,0.087396
2,661,34.434096,-3.757658,4.4956,0.1854,0.8957,0.0099,3.0743,0.1384,22.0689,...,661,0.111208,0,18.51870,657,1.987677,1.979659,1.982701,11.251713,0.032812
3,1036,34.432450,-3.749924,7.6350,0.9149,0.5144,0.0085,5.0309,0.4047,21.9548,...,1036,0.922889,0,17.43010,1032,1.982618,1.976072,1.981512,10.689293,0.096161
4,300,34.435031,-3.767948,2.2217,0.0403,0.7417,0.0139,2.5740,0.1577,22.4572,...,300,0.951805,0,16.91640,295,1.989074,1.984283,1.986897,10.968724,0.013495
5,920,34.435628,-3.753143,1.4499,0.0395,0.8452,0.0179,6.3884,0.4414,22.5358,...,920,0.595968,0,14.83810,917,1.963658,1.961974,1.962999,NaN,NaN
6,305,34.447153,-3.768007,11.3433,0.9366,0.4094,0.0070,7.3004,0.3759,22.0240,...,305,1.644519,0,11.84510,298,1.995379,1.990165,1.992867,10.822529,0.076894
7,1057,34.436886,-3.750173,3.3123,0.0364,0.5120,0.0081,1.6321,0.0645,22.7747,...,1057,0.925692,0,11.80230,1050,1.982959,1.974407,1.977232,10.581541,0.108340
8,1065,34.435917,-3.749538,4.5865,0.3128,0.3704,0.0405,0.9374,0.2062,22.7974,...,1065,0.964151,0,11.22480,1064,1.996971,1.983367,1.987974,10.741509,0.106439
9,608,34.438450,-3.760699,2.4748,0.0330,0.7395,0.0117,1.2098,0.0633,22.8834,...,608,0.488415,0,10.55160,606,1.971139,1.961619,1.965895,10.308117,0.118456


In [65]:
# Save
hst_savename = '/data/emiln/XLSSC122_GalPops/Analysis/Results/hst_galfit_full.csv'
mhst_mdf.to_csv(hst_savename,index=False)

### 3DHST

In [6]:
# Load catalogs
DHST_cat_root = '/data/emiln/XLSSC122_GalPops/Data/3DHST/Products/catalogs/'
DHST_cat_filename = DHST_cat_root+'cosmos_3dhst.v4.1.cats/Catalog/cosmos_3dhst.v4.1.cat'
phot_df = gfh.load_3DHST_galaxy_catalog(DHST_cat_filename,  mag='F140W', magthresh=24, z=2.00, z_thresh=0.1, overwrite=False, verbose=True, goodfit=True) 

galfit_filename = '/data/emiln/XLSSC122_GalPops/Analysis/3DHST/Results/3DHST_z2_full_data_cps_sigma_rms_meanexp_cps_w5HLR_F140W_PSF_5min_CONV64_CONSTR_DYNMAG.csv'
galfit_df = pd.read_csv(galfit_filename)


Checking if catalog exists at /data/emiln/XLSSC122_GalPops/Data/3DHST/Products/catalogs/magthresh24_z2.0_zthresh0.1.csv
Catalog already exists, returning DataFrame


In [9]:
phot_df.columns.values
# galfit_df.columns.values
phot_df['F814W'] = 25.0 - 2.5 * np.log10(phot_df['f_F814W'])

/data/emiln/miniconda2/lib/python2.7/site-packages/ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in log10
  This is separate from the ipykernel package so we can avoid doing imports until


In [10]:
# Merge
# cols_to_use = phot_df.columns.difference(galfit_df.columns)
dhst_mdf = galfit_df.merge(phot_df,how='inner',on='ID',suffixes=('','_y'))
dhst_mdf.drop(labels=['ra_y','dec_y'],axis=1,inplace=True)
print(dhst_mdf.columns.values)
print(len(dhst_mdf))
print(len(galfit_df))

['ID' 'ra' 'dec' 're' 're_err' 'ar' 'ar_err' 'n' 'n_err' 'mag' 'mag_err'
 'sky' 'sky_err' 'chi2nu' 'ErrorValue' 'id' 'x' 'y' 'faper_F160W'
 'eaper_F160W' 'faper_F140W' 'eaper_F140W' 'f_F160W' 'e_F160W' 'w_F160W'
 'f_U' 'e_U' 'w_U' 'f_B' 'e_B' 'w_B' 'f_G' 'e_G' 'w_G' 'f_V' 'e_V' 'w_V'
 'f_F606W' 'e_F606W' 'w_F606W' 'f_R' 'e_R' 'w_R' 'f_Rp' 'e_Rp' 'w_Rp'
 'f_I' 'e_I' 'w_I' 'f_Ip' 'e_Ip' 'w_Ip' 'f_F814W' 'e_F814W' 'w_F814W'
 'f_Z' 'e_Z' 'w_Z' 'f_Zp' 'e_Zp' 'w_Zp' 'f_UVISTA_Y' 'e_UVISTA_Y'
 'w_UVISTA_Y' 'f_F125W' 'e_F125W' 'w_F125W' 'f_J1' 'e_J1' 'w_J1' 'f_J2'
 'e_J2' 'w_J2' 'f_J3' 'e_J3' 'w_J3' 'f_J' 'e_J' 'w_J' 'f_UVISTA_J'
 'e_UVISTA_J' 'w_UVISTA_J' 'f_F140W' 'e_F140W' 'w_F140W' 'f_H1' 'e_H1'
 'w_H1' 'f_H2' 'e_H2' 'w_H2' 'f_H' 'e_H' 'w_H' 'f_UVISTA_H' 'e_UVISTA_H'
 'w_UVISTA_H' 'f_K' 'e_K' 'w_K' 'f_Ks' 'e_Ks' 'w_Ks' 'f_UVISTA_Ks'
 'e_UVISTA_Ks' 'w_UVISTA_Ks' 'f_IRAC1' 'e_IRAC1' 'w_IRAC1' 'f_IRAC2'
 'e_IRAC2' 'w_IRAC2' 'f_IRAC3' 'e_IRAC3' 'w_IRAC3' 'f_IRAC4' 'e_IRAC4'
 'w_IRAC4' 'f_IA427

In [11]:
# Save
dhst_savename = '/data/emiln/XLSSC122_GalPops/Analysis/Results/3dhst_galfit_full.csv'
dhst_mdf.to_csv(dhst_savename,index=False)